# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

Загрузим датафрейм и выведем его первые 5 строк

In [2]:
try:   
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
    
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Узнаем общую информацию про полученный датафрейм

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разделим исходные данные для обучения

In [4]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

In [5]:
<img src="https://emojigraph.org/media/apple/check-mark-button_2705.png" align=left width=33, heigth=33>
<div class="alert alert-success">
<b>v2</b> 👍 </div>features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345, stratify=target)
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345, stratify=target_train)

In [6]:
print('Обучающуя выборка:', features_train.shape, target_train.shape)
print('Валидационнуя выборка:', features_valid.shape, target_valid.shape)
print('Тестовая выборка:', features_test.shape, target_test.shape)

Обучающуя выборка: (1928, 4) (1928,)
Валидационнуя выборка: (643, 4) (643,)
Тестовая выборка: (643, 4) (643,)


Исходные данные разделены на 60%, 20% и 20%

## Исследуйте модели

Найдем accuracy наилучшей модели на валидационной выборке для дерева решений

In [7]:
best_model_tree = None
best_accuracy_tree = 0.75
best_depth_tree = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions_valid)
    if accuracy > best_accuracy_tree:
        best_model_tree = model
        best_accuracy_tree = accuracy
        best_depth_tree = depth

print(f'Лучший параметр Accuracy {best_accuracy_tree}, значение глубины дерева {best_depth_tree}')

Лучший параметр Accuracy 0.7993779160186625, значение глубины дерева 3


Найдем accuracy наилучшей модели на валидационной выборке для случайного леса

In [8]:
best_model_forest = None
best_accuracy_forest = 0.75
best_depth_forest = 0
best_est = 0
for est in range(1, 51):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy_forest:
            best_model_forest = model
            best_accuracy_forest = accuracy
            best_depth_forest = depth
            best_est = est

print(f'Лучший параметр Accuracy {best_accuracy_forest}, \
значение числа деревьев {best_est}, значение глубины дерева {best_depth_forest}')

Лучший параметр Accuracy 0.8242612752721618, значение числа деревьев 24, значение глубины дерева 10


Найдем accuracy наилучшей модели на валидационной выборке для логистической регресии 

In [9]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_regression.fit(features_train, target_train)
predictions_valid = model_regression.predict(features_valid)
accuracy_regression = accuracy_score(target_valid, predictions_valid)
print('Параметр Accuracy', accuracy_regression)

Параметр Accuracy 0.7371695178849145


Больше всего правильных ответов предсказала модель случайного леса с параметрами значения числа деревьев - 24 и значением глубины дерева - 10. Она имеет больше разных значений гиперпараметров которых можно изменить в ходе исследования, но из-за этого страдает скорость нахождения наилучшей модели, а также скорость работы. 

## Проверьте модель на тестовой выборке

Проверим качество наилучшей модели на тестовой выборке, а также объеденим обучающую и валидационную выборку

In [10]:
features_train = pd.concat([features_train, features_valid], ignore_index=True)
target_train = pd.concat([target_train, target_valid], ignore_index=True)
best_model_forest.fit(features_train, target_train)
predictions_test = best_model_forest.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
print('Параметр Accuracy', accuracy)

Параметр Accuracy 0.8180404354587869


Точность предсказаний упала примерно на 1%.

## Проверьте модели на адекватность

In [11]:
dummy_model = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_model.fit(features_train, target_train) 
predictions_valid_dummy_model = dummy_model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions_valid_dummy_model)
print('Accuracy Dummy модели для валидационной выборке',accuracy)
predictions_test_dummy_model = dummy_model.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test_dummy_model)
print('Accuracy Dummy модели для тестовой выборке',accuracy)


Accuracy Dummy модели для валидационной выборке 0.6936236391912908
Accuracy Dummy модели для тестовой выборке 0.6936236391912908


Если полученные предсказания выбранных моделей будут больше, чем полученные предсказания Dummy модели, то наши выбранные модели можно считать адекватными. 